# AERONET analysis

This is the code helps you to analysis and plot AERONET data. If you are not confident in Python, don't worry! You only need to make modifications to the statements found as follows:

#------------------------------------------------------------------------------------------------------------------------------

         Modify ONLY within here
#------------------------------------------------------------------------------------------------------------------------------

Important:
- A recurring error may be that paths are specified with '\\' as the folder separator, while the Python language uses '\\\\' (double).
- It is also recommended to run the cells one by one, and not to re-run a cell while it is executing (the software may hang). If while a cell is running, any changes are made to the code in that cell and we want to re-run it, we should wait for it to finish or restart the kernels.
- If desired, you can play with the rest of the code.

## IMPORTING LIBRARIES AND SPECIFYING PATHS

First, we need to import the necessary libraries. In case any of the modules are not found when running, install the module using:

#------------------------------------------------------------------------------------------------------------------------------

    !pip install <module_name>
#------------------------------------------------------------------------------------------------------------------------------


In [ ]:
from pathlib import Path
import numpy as np #For general mathematical calculations
from datetime import datetime
import matplotlib.pyplot as plt #For plotting
from scipy.signal import savgol_filter #Allows to smooth a profile through the Savitzky-Golay filter

#For working with aeronet data (developed by the Atmospheric Physics Group of the UGR, GFAT)
from gfatpy.aeronet.reader import reader, header
from gfatpy.aeronet import plot
from gfatpy.utils.plot import font_axes

%load_ext autoreload
%autoreload 2

%matplotlib inline

In the next cell, we define the directory where the lidar measurement files are located and the directory where we want to save the graphs.

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------
aeronet_dir = Path(r'C:\Users\Usuario\Mi unidad\04. Proyectos\ATMO-ACCESS\curso\data\aeronet')  #Database folder path, until '\1a' (included). Example: 'C:\\Users\\user\\Desktop\\data\\1a'
output_dir = Path(r'C:\Users\Usuario\Mi unidad\04. Proyectos\ATMO-ACCESS\curso\output')  #Output folder path. Example: ''C:\\Users\\user\\Desktop\\output''
#------------------------------------------------------------------------------------------------------------------------------

print([aeronet_dir.is_dir(), output_dir.is_dir()]) #Comprueba que ambas carpetas existen.


## LOAD AERONET DATA

AERONET data are read with `reader`. This program can read AERONET files with extentions `*.all` and `*.lev15`.

In [ ]:
aeronet_wildcard = (aeronet_dir /  '*Granada.all').absolute().as_posix()
header, aeronet = reader(aeronet_wildcard)

In [ ]:
#This is the header of the file
header

In [ ]:
#Available variables
[*aeronet.columns]

## Plot AOD time series

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------
# properties2plot = ['AOD_Coincident_Input[440nm]'] # properties to plot in a same axis
properties2plot = 'all' # if 'all' plots the AODs at the available wavelengths in a same axis
ylims = (0,0.5) #Y axis limits 
#------------------------------------------------------------------------------------------------------------------------------
if properties2plot == 'all':
    properties2plot = [key_ for key_ in aeronet.keys() if key_.find("AOD_Coincident_Input") != -1 and not (aeronet[key_].values == -999).all()]
print(properties2plot)
_, _, figure_pathname = plot.aod(
    aeronet,
    header,
    properties2plot,
    datetime(2023, 1, 1),
    datetime(2023, 5, 1),
    figure_dir=output_dir,
    ylims = ylims,
)

## Plot Angstrom exponent time series

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------
# properties2plot = ['AOD_Coincident_Input[440nm]'] # properties to plot in a same axis
properties2plot = 'Angstrom_Exponent_440-870nm_from_Coincident_Input_AOD' # if 'all' plots the AODs at the available wavelengths in a same axis
ylims = (0,0.5) #Y axis limits 
filename= 'ae.png'
dpi_fig = 300
#------------------------------------------------------------------------------------------------------------------------------

fig, axes = plt.subplots(1, 1, figsize=[10, 5])
aeronet['Angstrom_Exponent_440-870nm_from_Coincident_Input_AOD'].plot(marker="o", markersize=5, linewidth=0, ax=axes)
axes.set_ylabel("AE[440-870 nm], [#]")
axes.grid()

figure_pathname = output_dir / filename
plt.savefig(figure_pathname, dpi=dpi_fig, bbox_inches="tight")

## Plot particle size distribution

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------
# properties2plot = ['AOD_Coincident_Input[440nm]'] # properties to plot in a same axis
date_str = '2023-02-27' # if 'all' plots the AODs at the available wavelengths in a same axis
filename= 'psd.png'
type_distribution='volume'
fig_size = (8,6)
dpi_fig = 300
#------------------------------------------------------------------------------------------------------------------------------

fig, axes, figurepath = plot.distribution(aeronet, date_str,type_distribution=type_distribution, fig_size=fig_size, dpi=dpi_fig,figure_dir=output_dir)
